# ATRX Mutation Trans Effect on Transcriptomics

### Library Imports

In [1]:
import pandas as pd
import numpy as np
import scipy.stats

import warnings
warnings.filterwarnings("ignore")

import cptac
import cptac.utils as al

brain = cptac.Gbm()

### Select Gene

In [2]:
gene = "ATRX"

### Investigate Proteomics, Phosphoproteomics, Acetylproteomics, or Transcriptomics

In [3]:
#omics = "proteomics"
omics = "transcriptomics"
#omics = "phosphoproteomics"
#omics = "acetylproteomics"

### Track all significant comparisons in Dataframe

In [4]:
all_significant_comparisons = pd.DataFrame(columns=['Cancer_Type', 'Gene', 'Comparison','Interacting_Protein','P_Value'])

In [5]:
def add_to_all_significant_comparisons(df, cancer, interacting, all_sig_comp):
    expanded = df
    expanded['Gene'] = gene
    expanded['Cancer_Type'] = cancer
    expanded['Interacting_Protein'] = interacting
    
    updated_all_comparisons = pd.concat([all_sig_comp, expanded], sort=False)
    
    return updated_all_comparisons

# Interacting Proteins: Transcriptomics

### Generate interacting protein list

Make a call to the cptac.algorithms get interacting proteins method, which interacts with the uniprot and string databases to generate a list of known interacting partners with the given gene

In [6]:
'''Use get interacting proteins method to generate list of interacting proteins'''
interacting_proteins = al.get_interacting_proteins(gene)

'''Show interacting protein list'''
print("Interacting Proteins:")
for interacting_protein in interacting_proteins:
    print(interacting_protein)

Interacting Proteins:
NKTR
PNN
TP53
STAG2
H3F3B
SETDB1
NIPBL
PHIP
BPTF
RSF1
PRKDC
MUS81
ARID4A
RAD52
BDP1
XRCC6
SMC5
H3F3A
TPR
SCAF11
ATRX
DAXX
RAD51
MECP2
XRCC3
HIST2H3PS2
EIF4A2
EZH2
Mecp2
CBX5


### Test for significant comparisons in any of interacting proteins

In [7]:
'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = brain.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins)
protdf = brain.reduce_multiindex(protdf, flatten=True)
protdf = protdf[protdf.Sample_Status == 'Tumor'] # drop Normal samples

'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

'''Format the dataframe correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)


'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")
        
        all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Gbm", True, all_significant_comparisons)
        

Doing t-test comparisons

                                 Comparison   P_Value
0  MECP2_transcriptomics_ENSG00000169057.18  0.000466





# All Proteins: Transcriptomics 

In [8]:
try:
    print("\nGene: ", gene)

    # Use all proteins

    # Create dataframe in order to do comparisons with wrap_ttest
    protdf = brain.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics)
    protdf = brain.reduce_multiindex(protdf, flatten=True) # flatten for specific column names
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    # Create binary column 
    protdf['Label'] = np.where(
                protdf[gene+'_Mutation_Status'] == 'Wildtype_Tumor', 'Wildtype', 'Mutated')

    # Format the dataframe correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    # Make list of columns to be compared using t-tests
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    # Call wrap_ttest, pass in formatted dataframe
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    # Print results, if anything significant was found
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")
            
            all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Gbm", False, all_significant_comparisons)


except Exception as e:
    print("Error in Comparison")
    print(e)


Gene:  ATRX


Doing t-test comparisons

                                         Comparison       P_Value
0         SNORD23_transcriptomics_ENSG00000221139.1  2.285907e-12
1         MIR4311_transcriptomics_ENSG00000263512.1  3.003773e-10
2         TSPYL2_transcriptomics_ENSG00000184205.13  1.119606e-09
3   RP11-296E23.2_transcriptomics_ENSG00000265948.1  1.550183e-09
4    RP11-624L4.1_transcriptomics_ENSG00000259345.4  2.222779e-09
5    RP11-80F22.5_transcriptomics_ENSG00000259836.1  3.792065e-09
6    IGHD4OR15-4A_transcriptomics_ENSG00000271317.1  4.108837e-09
7     RP1-249C1.1_transcriptomics_ENSG00000271157.1  5.404823e-09
8         CCDC68_transcriptomics_ENSG00000166510.12  7.968308e-09
9           OR4M1_transcriptomics_ENSG00000176299.4  8.107254e-09
10        GREB1L_transcriptomics_ENSG00000141449.13  1.818423e-08
11  RP11-296E23.1_transcriptomics_ENSG00000265751.1  3.027580e-08
12       RNU7-73P_transcriptomics_ENSG00000239105.1  4.230296e-08
13   RP11-80F22.7_transcriptomics_ENSG00000260153.

# Print all significant comparisons

In [9]:
if len(all_significant_comparisons) > 0:
    display(all_significant_comparisons)
    
else:
    print('No Significant Comparisons!')

,Cancer_Type,Gene,Comparison,Interacting_Protein,P_Value
0,Gbm,ATRX,MECP2_transcriptomics_ENSG00000169057.18,True,4.663027e-04
0,Gbm,ATRX,SNORD23_transcriptomics_ENSG00000221139.1,False,2.285907e-12
1,Gbm,ATRX,MIR4311_transcriptomics_ENSG00000263512.1,False,3.003773e-10
2,Gbm,ATRX,TSPYL2_transcriptomics_ENSG00000184205.13,False,1.119606e-09
3,Gbm,ATRX,RP11-296E23.2_transcriptomics_ENSG00000265948.1,False,1.550183e-09
4,Gbm,ATRX,RP11-624L4.1_transcriptomics_ENSG00000259345.4,False,2.222779e-09
5,Gbm,ATRX,RP11-80F22.5_transcriptomics_ENSG00000259836.1,False,3.792065e-09
6,Gbm,ATRX,IGHD4OR15-4A_transcriptomics_ENSG00000271317.1,False,4.108837e-09
7,Gbm,ATRX,RP1-249C1.1_transcriptomics_ENSG00000271157.1,False,5.404823e-09
8,Gbm,ATRX,CCDC68_transcriptomics_ENSG00000166510.12,False,7.968308e-09


### Write Significant Comparisons (if any) to Shared CSV file

In [10]:
'''
existing_results = pd.read_csv(gene+'_Trans_Results.csv')

updated_results = pd.concat([existing_results, all_significant_comparisons], sort=False)

updated_results.to_csv(path_or_buf = gene + '_Trans_Results.csv', index=False)'''

"\nexisting_results = pd.read_csv(gene+'_Trans_Results.csv')\n\nupdated_results = pd.concat([existing_results, all_significant_comparisons], sort=False)\n\nupdated_results.to_csv(path_or_buf = gene + '_Trans_Results.csv', index=False)"